# ReAct Agent Implementation

## Overview
In this example we will guide you through how to create a ReAct (Reasoning and Acting) agent implementation using Strands Agents default orchestration. We will demonstrate a single-agent system that combines reasoning with direct tool execution, providing a baseline for comparison with multi-agent orchestration patterns. This notebook also serves as an introduction to TauBench tools and dataset for airline customer service scenarios.

## Architecture

<div style="text-align:center">
    <img src="../images/react.png" alt="ReAct Agent Architecture" width="600">
    <p>The system consists of a single agent that handles both reasoning and action:</p>
    <p><em>ReAct Architecture: User Query → [ReAct Agent] → Final Response</em></p>
</div>

## Key Features

* **Default Strands Agents orchestration**: Uses built-in agent orchestration without custom multiagent graphs
* **TauBench integration**: Demonstrates airline tools and dataset usage for realistic scenarios
* **Single-agent simplicity**: Direct reasoning-action loops without orchestration complexity
* **Tool integration**: Full access to airline tools suite for comprehensive task handling
* **Baseline comparison**: Provides performance baseline against multi-agent patterns

## Agent Details
<div style="float: left; margin-right: 20px;">
    
|Feature             |Description                                        |
|--------------------|---------------------------------------------------|
|Native tools used   |Full airline tools suite (14 tools)             |
|Custom tools created|None (uses existing MAbench/TauBench tools)      |
|Agent Structure     |Single-agent with default Strands Agents orchestration |
|AWS services used   |Amazon Bedrock                                   |

</div>


In [ ]:
!pip3 install -r ./requirements.txt --quiet 
!pip3 install strands-agents==v1.6.0 strands-agents-tools==v0.2.5 --quiet
!pip3 install litellm --quiet

## Importing dependency packages

Now let's import all the necessary libraries and modules for our Reflexion implementation. This includes standard Python libraries, AWS SDK components, Strands Agents framework modules, and custom helper functions.

In [ ]:
# Libraries

import sys
sys.path.append('../data/ma-bench/')
sys.path.append('../data/tau-bench/')

import os
import json
import logging

# Strands imports
from strands import Agent
from strands.models import BedrockModel

## Import airline domain tools

Now we'll import the comprehensive set of airline domain tools from MAbench and TauBench. These tools provide the actual functionality that our ReAct agent will execute, including flight booking, reservation management, and customer service operations.

In [ ]:
from mabench.environments.airline.tools.book_reservation import book_reservation
from mabench.environments.airline.tools.calculate import calculate
from mabench.environments.airline.tools.cancel_reservation import cancel_reservation
from mabench.environments.airline.tools.get_reservation_details import get_reservation_details
from mabench.environments.airline.tools.get_user_details import get_user_details
from mabench.environments.airline.tools.list_all_airports import list_all_airports
from mabench.environments.airline.tools.search_direct_flight import search_direct_flight
from mabench.environments.airline.tools.search_onestop_flight import search_onestop_flight
from mabench.environments.airline.tools.send_certificate import send_certificate
from mabench.environments.airline.tools.think import think
from mabench.environments.airline.tools.transfer_to_human_agents import transfer_to_human_agents
from mabench.environments.airline.tools.update_reservation_baggages import update_reservation_baggages
from mabench.environments.airline.tools.update_reservation_flights import update_reservation_flights
from mabench.environments.airline.tools.update_reservation_passengers import update_reservation_passengers

domain = "airline"

from tau_bench.envs.airline.wiki import WIKI

## Configure Strands Agents

Now let's set up the core Strands Agents components that will power our ReAct  system. We need to configure the Amazon Bedrock connection, conversation management, and logging to ensure our three-agent pipeline runs smoothly.

### Framework Setup Process

First, we'll establish the **AWS region** and create a `BedrockModel` instance that will be used by our agent. We do this so that all agents use the same LLM configuration for consistent behavior. 

Finally, we'll configure **logging** to minimize noise during execution so we can focus on the react execution flow and results.

In [ ]:
# Create BedrockModel with specified region
region="us-east-1"
bedrock_model_taubench = BedrockModel(region_name= region)

#setup logging
# Disable all logging except critical errors
logging.basicConfig(level=logging.CRITICAL)

# Silence specific noisy loggers completely
for logger_name in ["strands", "graph", "event_loop", "registry", "sliding_window_conversation_manager", "bedrock", "streaming"]:
    logging.getLogger(logger_name).setLevel(logging.CRITICAL)


## ReAct Agent Implementation

Now let's implement the **ReAct (Reasoning and Acting) agent** that combines reasoning, tool execution, and response synthesis in a single agent loop. This represents the traditional approach where one agent handles all aspects of task completion through iterative think-act-observe cycles.

### Airline Tools Configuration

The ReAct agent has access to the complete suite of 14 airline domain tools, enabling it to handle any customer service scenario. We provide this comprehensive toolset so that the agent can directly execute booking operations, search flights, update reservations, and manage passenger information without requiring separate planning or execution phases.

### System Prompt Design

We will first create a specialized system prompt for airline customer service with specific instructions for handling geographic information and airport locations. We include policy guidance from the WIKI so that the agent understands airline business rules, fare classes, and operational constraints while maintaining helpful customer service behavior.

The prompt emphasizes avoiding placeholder arguments and properly inferring U.S. state information for airports, ensuring accurate tool execution with real data rather than fabricated values.

### ReAct Agent 

The `llm_react_agent()` function combines the system prompt, model configuration, and tool suite into a complete Strands Agent. This creates a single-agent system that can handle complex multi-step airline scenarios through ReAct's iterative reasoning process, providing a baseline for comparison with the orchestrated multi-agent patterns.

The ReAct approach demonstrates direct tool execution with reasoning, where the agent autonomously decides which tools to call, when to call them, and how to synthesize the results into customer-friendly responses.




In [ ]:
tools = [
    book_reservation,
    calculate,
    cancel_reservation,
    get_reservation_details,
    get_user_details,
    list_all_airports,
    search_direct_flight,
    search_onestop_flight,
    send_certificate,
    think,
    transfer_to_human_agents,
    update_reservation_baggages,
    update_reservation_flights,
    update_reservation_passengers,
]

system_prompt_template = """
You are a helpful assistant for a travel website. Help the user answer any questions.

<instructions>
- Remeber to check if the the airport city is in the state mentioned by the user. For example, Houston is in Texas.
- Infer about the the U.S. state in which the airport city resides. For example, Houston is in Texas.
- You should not use made-up or placeholder arguments.
<instructions>

<policy>
{policy}
</policy>
"""

prompt = system_prompt_template.replace("{policy}", WIKI)

def llm_react_agent(tools):

    return Agent( 
        model = bedrock_model_taubench, 
        tools = tools, 
        system_prompt = prompt,
       
    )

## Load Dataset

Now let's load the **TauBench evaluation dataset** that contains real airline customer service scenarios. We do this so that we can test our ReAct agent against standardized benchmarks and measure its performance on authentic customer queries like:

- Flight changes
- Cancellations  
- Booking modifications

This loads the **single-turn airline tasks** from TauBench, which provides us with a collection of customer queries along with their expected outcomes for evaluation purposes.

In [ ]:
output_path = os.path.join("..", "data", "tau-bench", "tau_bench", "envs", f"{domain}", "tasks_singleturn.json")
with open(output_path, "r") as file:
    tasks = json.load(file)


In [ ]:
import time
def test_react_graph(task, question_id):
    user_id = task['user_id']    
    user_query= task["question"]
    print(user_query)    
    agent = llm_react_agent(tools=tools)
    start=time.time()
    react_response = agent(user_query)    
    exec_time=time.time()-start
    print(f" REACT execution time {exec_time} \n")
    
question_id = 20 #20, #48 #43 #39
task = tasks[question_id]   
test_react_graph(task, question_id)

##NOTE: To run other queries please rerun all the other cells

# Sample ReAct Framework Execution 

## User Query
"My user id is james_taylor_7043. I want to change my upcoming one-stop flight from LAS to IAH to a nonstop flight. My reservation ID is 1N99U6. I also want to remove my checked bag and want the agent to refund me for the same."

## ReAct Framework Response

### Reasoning and Acting Phase
**Action 1**: `get_reservation_details` → Retrieved current booking with one-stop outbound flight
**Action 2**: `search_direct_flight` → Found HAT175 ($155) and HAT266 ($146) direct options

**Final Response**: 
> **Direct Flight Options:**
> 1. HAT175 - Departs 5:00 PM ($155, save $198)
> 2. HAT266 - Departs 1:00 PM ($146, save $207)
> 
> **Baggage Policy**: Unfortunately, our policy does not allow removing checked bags after booking
> 
> Which direct flight would you prefer?

### Execution Summary
- **Total execution time**: 13.64 seconds
- **Tools used**: 2 tools only
- **Status**: Incomplete - requesting user decision without proper escalation

## Critical Failure: No Human Agent Transfer

### What ReAct Failed to Do (vs Other Frameworks)

**Missing Human Escalation**: Unlike ReWOO, Reflexion, and Hybrid frameworks that all recognized the need to transfer baggage removal requests to human agents, ReAct completely **failed to escalate** this issue.

**Comparison with Other Frameworks:**
- **ReWOO**: Planned and executed `transfer_to_human_agents` for baggage removal
- **Reflexion**: Recommended human agent transfer for policy verification and complex calculations  
- **Hybrid**: Included human transfer in the structured plan for baggage assistance
- **ReAct**: **No transfer attempted** - left user with unresolved baggage request

## Congrats!

Congratulations! You've successfully created and tested a ReAct agent implementation using Strands Agents default orchestration. This system demonstrates:

- **Single-agent architecture** using Strands Agents built-in orchestration for direct reasoning-action loops without custom multiagent complexity
- **TauBench tools integration** with comprehensive airline tools suite (booking, search, updates) for real-world customer service scenarios
- **Dataset exploration** using TauBench evaluation datasets to understand airline domain tasks and performance benchmarking
- **Baseline performance** providing a foundation for comparing against more complex orchestration patterns

The ReAct pattern excels at straightforward tasks requiring direct tool interaction and adaptive reasoning, making it ideal for simple airline queries and serving as a performance baseline. In the next notebooks, we will explore custom orchestration patterns like REWOO and Reflexion that build upon this foundation to handle more complex multi-step scenarios requiring specialized agent coordination.